In [ ]:
import pandas as pd
import re
import numpy as np
from collections import Counter
from rapidfuzz import fuzz, process

# Load the HERO CSV file
hero_data = pd.read_csv('/home/bipin/Documents/kotak/car_model/manufacturer_data/HERO.csv')
print(f"Loaded {len(hero_data)} records from HERO.csv")
hero_data.head()

# Function to clean and normalize model names
def clean_model(text):
    if pd.isna(text) or text is None:
        return ""
    # Convert to string
    text = str(text).upper()
    # Remove common prefixes and company names
    text = re.sub(r'HERO\s+HONDA\s+MOTORS\s+LTD,?\s*', '', text)
    text = re.sub(r'HERO\s+MOTOCORP\s+LTD,?\s*', '', text)
    text = re.sub(r'HERO\s+MOTO\s+CORP\s+LTD\.?,?\s*', '', text)
    text = re.sub(r'HERO\s+HONDA\s+MOTORS,?\s*', '', text)
    text = re.sub(r'HERO\s+HONDA,?\s*', '', text)
    text = re.sub(r'HERO\s+ELECTRIC\s+VEHICLE\s+PVT\s+LTD,?\s*', '', text)
    text = re.sub(r'H\.\s*HONDA\s+', '', text)
    text = re.sub(r'H\.\s*H\.\s*', '', text)
    text = re.sub(r'\"', '', text)
    
    # Remove special characters and extra spaces
    text = re.sub(r'[^\w\s\./\-]', ' ', text)  # Keep ., /, and -
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create a list of cleaned model names
hero_data['clean_model'] = hero_data['rc_maker_model'].apply(clean_model)

# Filter out too generic or empty model names
generic_terms = ['BS', 'IV', 'III', 'BS-IV', 'BS-III', 'BSIV', 'BSIII', 'BS6', 'BSVI', 'LIMITED', 'LTD', 'NOT AVAILABLE', 'OTHER']
filtered_models = hero_data['clean_model'].apply(
    lambda x: x if x and all(term not in x.split() for term in generic_terms) else np.nan
)

# Count model name frequencies
model_counts = Counter([m for models in filtered_models.dropna() for m in models.split()])
model_counts = {k: v for k, v in model_counts.items() if v >= 3}  # Keep only somewhat frequent terms

# Print top model keywords by frequency
print("\nTop model keywords by frequency:")
sorted_counts = sorted(model_counts.items(), key=lambda x: x[1], reverse=True)
for model, count in sorted_counts[:30]:
    print(f"{model}: {count}")

# Define primary HERO model keywords
hero_model_keywords = [
    # Motorcycle Models
    "SPLENDOR", "SPLENDOR+", "SPLENDOR PLUS", "SPLENDOR PRO", "SPLENDOR NXG", "SPLENDOR ISMART", "SPLENDOR XTEC",
    "PASSION", "PASSION PRO", "PASSION PLUS", "PASSION XPRO",
    "GLAMOUR", "GLAMOUR XTEC", 
    "HF", "HF DELUXE", "HF DELUX",
    "CD", "CD DELUXE", "CD100", "CD DAWN",
    "SUPER SPLENDOR",
    "XTREME", "XTREME 160R", "XTREME 200R", "XTREME 200S",
    "XPULSE", "XPULSE 200", "XPULSE 200T",
    "HUNK", "IMPULSE", "ACHIEVER", "IGNITOR", "KARIZMA", "PLEASURE",
    
    # Scooter Models
    "MAESTRO", "MAESTRO EDGE", "DUET", "PLEASURE", "DESTINI", "PLEASURE+",
    
    # Electric Models
    "VIDA", "OPTIMA",
    
    # Features and Variants
    "SELF", "DRUM", "DISC", "CAST", "SPOKE", "WHEELS", "KICK", "BRAKE", "I3S",
    "CAST WHEELS", "SPOKE WHEELS", "SELF START", "KICK START",
    "REAR DISC", "DRUM BRAKE", "DISC BRAKE", "XTEC", "FI", "SMART"
]

# Create alias map for common misspellings and variations
hero_alias_map = {
    # Model Aliases
    "SPLND": "SPLENDOR",
    "SPLND.": "SPLENDOR",
    "SPELENDOR": "SPLENDOR",
    "SPLENDOUR": "SPLENDOR",
    "SPLENDER": "SPLENDOR",
    "SPLEDOR": "SPLENDOR",
    "SPLENDOR +": "SPLENDOR+",
    "SPLENDOR PLUS": "SPLENDOR+",
    "SPLENDOR - PRO": "SPLENDOR PRO",
    "SPLENDOR - NXG": "SPLENDOR NXG",
    "SPLENDOR I-SMART": "SPLENDOR ISMART",
    "SPLEDOR ISMART": "SPLENDOR ISMART",
    
    "PAP": "PASSION",
    "PPR": "PASSION PRO",
    "PASSION PLUS": "PASSION+",
    "PASSION +": "PASSION+",
    
    "HF DELUX": "HF DELUXE",
    "HF DELUXE CAST": "HF DELUXE",
    
    "CD-DELUXE": "CD DELUXE",
    "CD DELUXE CAST SELF": "CD DELUXE",
    "CD DELUXE SELF CAST": "CD DELUXE",
    "CD DELUXE BS II CAST WHEELS": "CD DELUXE",
    "CD DELUXE SPOKE WHEELS": "CD DELUXE",
    
    "SUPER SPLENDOR CAST": "SUPER SPLENDOR",
    "SUPER SPLENDOR DR SELF CAST": "SUPER SPLENDOR",
    "SUPER SPLENDOR DRM/SLF-CASTWHL": "SUPER SPLENDOR",
    
    "GLAMOUR-DRUM-SELF-CAST-S": "GLAMOUR",
    "GLAMOUR-DISC-SELF(CAST WHEELS)": "GLAMOUR DISC",
    "GLAMOUR- XTEC-DISC-SELF-CAST": "GLAMOUR XTEC",
    "GLAMOUR DRUM BRAKE": "GLAMOUR",
    
    "HUNK ES REAR DISC BRK": "HUNK",
    "HUNK NEW": "HUNK",
    
    "XTREME REAR DRUM BRAKE": "XTREME",
    "XTREME 160R-DISC-DISC-CAST(SE)": "XTREME 160R",
    
    "MAESTRO EDGE 125 FI": "MAESTRO EDGE 125",
    
    # Feature Aliases
    "DR SELF CAST": "DRUM SELF CAST",
    "DRUM CAST": "DRUM CAST WHEELS",
    "DISC CAST": "DISC CAST WHEELS",
    "DRUM SELF CAST": "DRUM SELF CAST WHEELS",
    "DISC SELF CAST": "DISC SELF CAST WHEELS",
    "DRUM KICK CAST": "DRUM KICK CAST WHEELS",
    "DISC KICK CAST": "DISC KICK CAST WHEELS",
    "CAST WHEEL": "CAST WHEELS",
    "SPOKE WHEEL": "SPOKE WHEELS",
    "CARBO": "CARBON",
    "DRK": "DRUM KICK",
    "DRN": "DRUM",
    "SLF": "SELF",
    "CCR": "CARBON"
}

# Function to normalize text
def normalize(text):
    # Handle NaN/None values or any non-string types
    if pd.isna(text) or text is None:
        return ""
    # Convert to string if it's not already a string
    if not isinstance(text, str):
        text = str(text)
    # Then proceed with normalization
    return re.sub(r'[^A-Z0-9 \.]', '', text.upper())

# Function to extract Hero model
def extract_hero_model(text):
    txt = normalize(text)
    
    # Skip entries that clearly aren't Hero motorcycles
    # These are car models that appear in the data due to errors
    car_models = ["AMBASSADOR", "LANCER", "I20", "HONDA CITY", "HONDA SIEL"]
    for car in car_models:
        if car in txt:
            return "NOT FOUND"
    
    # Fix aliases (correct misspellings and other variations)
    for bad, good in hero_alias_map.items():
        if bad in txt:
            txt = txt.replace(bad, good)
    
    # Check for specific model families
    if any(s in txt for s in ["SPLENDOR", "SPLND", "SPLENDER"]):
        if "ISMART" in txt or "I SMART" in txt:
            return "SPLENDOR ISMART"
        if "XTEC" in txt:
            return "SPLENDOR+ XTEC"
        if "PRO" in txt:
            return "SPLENDOR PRO"
        if "NXG" in txt:
            return "SPLENDOR NXG"
        if "+" in txt or "PLUS" in txt:
            return "SPLENDOR+"
        return "SPLENDOR"
    
    if "SUPER SPLENDOR" in txt:
        return "SUPER SPLENDOR"
    
    if "PASSION" in txt or "PPR" in txt or "PAP" in txt:
        if "PRO" in txt:
            return "PASSION PRO"
        if "PLUS" in txt or "+" in txt:
            return "PASSION+"
        if "XPRO" in txt:
            return "PASSION XPRO"
        return "PASSION"
        
    if "GLAMOUR" in txt:
        if "XTEC" in txt:
            return "GLAMOUR XTEC"
        if "DISC" in txt:
            return "GLAMOUR DISC"
        return "GLAMOUR"
        
    if "HF" in txt or "DELUXE" in txt or "DELUX" in txt:
        return "HF DELUXE"
        
    if "CD" in txt:
        if "DELUXE" in txt or "DELUX" in txt:
            return "CD DELUXE"
        if "100" in txt:
            return "CD100"
        if "DAWN" in txt:
            return "CD DAWN"
        return "CD"
        
    if "XTREME" in txt:
        if "160R" in txt:
            return "XTREME 160R"
        if "200R" in txt:
            return "XTREME 200R"
        if "200S" in txt:
            return "XTREME 200S"
        return "XTREME"
        
    if "XPULSE" in txt:
        if "200T" in txt:
            return "XPULSE 200T"
        if "200" in txt:
            return "XPULSE 200"
        return "XPULSE"
        
    if "HUNK" in txt:
        return "HUNK"
        
    if "MAESTRO" in txt:
        if "EDGE 125" in txt:
            return "MAESTRO EDGE 125"
        if "EDGE" in txt:
            return "MAESTRO EDGE"
        return "MAESTRO"
        
    if "PLEASURE" in txt:
        if "+" in txt:
            return "PLEASURE+"
        return "PLEASURE"
        
    if "DESTINI" in txt:
        return "DESTINI"
        
    if "ACHIEVER" in txt:
        return "ACHIEVER"
        
    if "KARIZMA" in txt:
        return "KARIZMA"
        
    if "IMPULSE" in txt:
        return "IMPULSE"
        
    if "IGNITOR" in txt:
        return "IGNITOR"
        
    if "DUET" in txt:
        return "DUET"
    
    if "JOYRST" in txt or "JOY" in txt:
        return "JOY"
        
    if "VIDA" in txt:
        return "VIDA"
        
    if "OPTIMA" in txt:
        return "OPTIMA"
    
    # Check for any other keywords from our list
    for keyword in sorted(hero_model_keywords, key=len, reverse=True):
        if keyword in txt:
            return keyword
    
    # If nothing matches, use fuzzy matching as fallback
    try:
        # Use rapidfuzz for fuzzy matching
        from rapidfuzz import process
        
        # Try to find the best match with a minimum score
        result = process.extractOne(txt, hero_model_keywords, scorer=fuzz.token_set_ratio)
        if result and result[1] >= 80:  # if similarity score is at least 80%
            return result[0]
    except Exception:
        # If rapidfuzz isn't available or fails, we'll just continue
        pass
    
    return "NOT FOUND"

# Apply the extraction to the dataset
hero_data['hero_model'] = hero_data['rc_maker_model'].apply(extract_hero_model)

# Count the frequency of each extracted model
model_distribution = hero_data['hero_model'].value_counts()
print("\nExtracted model distribution:")
print(model_distribution.head(20))

# Create final model column with fallback logic
hero_data['final_model'] = hero_data['hero_model']
hero_data.loc[hero_data['final_model'] == "NOT FOUND", 'final_model'] = "HERO OTHER"

# Save the processed data
hero_data.to_csv('/home/bipin/Documents/kotak/car_model/clean_model/hero_processed.csv', index=False)

# Save the model mapping for verification and record
model_mapping = pd.DataFrame({
    'original_model': hero_data['rc_maker_model'].unique()
})
model_mapping['cleaned_model'] = model_mapping['original_model'].apply(clean_model)
model_mapping['extracted_model'] = model_mapping['original_model'].apply(extract_hero_model)
model_mapping['final_model'] = model_mapping['extracted_model']
model_mapping.loc[model_mapping['final_model'] == "NOT FOUND", 'final_model'] = "HERO OTHER"

model_mapping.to_csv('/home/bipin/Documents/kotak/car_model/clean_model/hero_model_mapping.csv', index=False)

print("\nProcessing complete. Files saved:")
print("1. Processed data: hero_processed.csv")
print("2. Model mapping: hero_model_mapping.csv")

# Print some statistics
print(f"\nTotal records: {len(hero_data)}")
print(f"Unique original models: {len(hero_data['rc_maker_model'].unique())}")
print(f"Unique extracted models: {len(hero_data['hero_model'].unique())}")
print(f"Records mapped to known models: {(hero_data['hero_model'] != 'NOT FOUND').sum()}")
print(f"Records not mapped (HERO OTHER): {(hero_data['hero_model'] == 'NOT FOUND').sum()}")
print(f"Mapping success rate: {(hero_data['hero_model'] != 'NOT FOUND').sum() / len(hero_data) * 100:.2f}%")